In [63]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import math
from tqdm import trange

# Read summary file

In [64]:
fp_summary = '../Exported_data/USGS_SHARE_02.20.22.csv'
def read_summary(fp_summary):
    df_vedio_summary = pd.read_csv(fp_summary)
    this_vedio_index = 1 # the row number of the video
    df_vedio_summary = df_vedio_summary.loc[this_vedio_index].to_frame().T
    return df_vedio_summary
df_vedio_summary = read_summary(fp_summary)

# Read template file

In [65]:
fp_template = '../Data_Template/data_template.csv'
def read_template(fp_template):
    df_template = pd.read_csv(fp_template)
    return df_template
df_template = read_template(fp_template)

# Read annotation file

# Shake

In [ ]:
fp_annotation = '../02/shake/02_Simplified_Annotated_Anchorage_Nestbett_Courthouse_2nd_Floor_Jozie.txt'
data = read_annotation(fp_annotation)
df_template = read_template(fp_template)

In [67]:
df_behavior = data[data['Earthquake Intensity'].isnull() == False]\
                .reset_index()
t_last_behavior_end = df_behavior['End Time - ss.msec'].max()
#df_behavior['End Time - ss.msec'].idxmax(axis = 0)
idx_reserved_max = t_last_behavior_end
df_template = df_template.drop(np.arange(idx_reserved_max, 500))


for i in trange(len(df_behavior)):
    t_curr_state_begin = df_behavior.loc[i, 'Begin Time - ss.msec'] 
    t_curr_state_end = df_behavior.loc[i, 'End Time - ss.msec']
    
    df_template.loc[t_curr_state_begin:t_curr_state_end, 'shaking_intensity'] = \
                        df_behavior.loc[i, 'Earthquake Intensity']
    
#df_template['behavioral_state'] = df_template['behavioral_state'].fillna(method = 'ffill')
shake = df_template.copy()[['shaking_intensity','time_elapsed']]

100%|██████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 3061.16it/s]


In [71]:
fp_annotation = '../02/02_Individual_Level_Leader_Annotated_Anchorage_Nestbett_Courthouse_2nd_Floor_Jozie.txt'
df_template = read_template(fp_template)
def read_annotation(fp_annotation):
    
    data = pd.read_csv(fp_annotation,
                       sep='\t') \
            .drop(['Begin Time - hh:mm:ss.ms', 'End Time - hh:mm:ss.ms','Duration - hh:mm:ss.ms'],
                  axis = 1)
    print(data.columns)
    
    data['Begin Time - ss.msec'] = round(data['Begin Time - ss.msec'])
    data['End Time - ss.msec'] = round(data['End Time - ss.msec'])
    data['Duration - ss.msec'] = data['End Time - ss.msec'] - data['Begin Time - ss.msec']
    
    data = data.sort_values(by = ['Begin Time - ss.msec', 'End Time - ss.msec'])
    
    return data

data = read_annotation(fp_annotation)

Index(['Begin Time - ss.msec', 'End Time - ss.msec', 'Duration - ss.msec',
       'Transcription', 'Phase', 'Triggers',
       'Responses/Self-protective Actions/Other-protective Actions',
       'Setting: Social and Physical', 'Injuries', 'Visible',
       'Earthquake Attributes', 'Participant 1 (P1)', 'Participant 2 (P2)',
       'Participant 3 (P3)'],
      dtype='object')


# Leader #1

In [72]:
df_behavior = data[data['Responses/Self-protective Actions/Other-protective Actions'].isnull() == False]\
                .reset_index()
t_last_behavior_end = df_behavior['End Time - ss.msec'].max()
#df_behavior['End Time - ss.msec'].idxmax(axis = 0)
idx_reserved_max = t_last_behavior_end
df_template = df_template.drop(np.arange(idx_reserved_max, 500))


for i in trange(len(df_behavior)):
    t_curr_state_begin = df_behavior.loc[i, 'Begin Time - ss.msec'] 
    t_curr_state_end = df_behavior.loc[i, 'End Time - ss.msec']
    
    df_template.loc[t_curr_state_begin:t_curr_state_end, 'behavioral_state'] = \
                        df_behavior.loc[i, 'Responses/Self-protective Actions/Other-protective Actions']
    
df_template['behavioral_state'] = df_template['behavioral_state'].fillna(method = 'ffill')


100%|████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<?, ?it/s]


In [79]:
df_template['num_people'] = df_vedio_summary['NumPeople'].values[0]
df_template['cover_availability'] = 1
df_template['child_present'] = 0 # 0 no 1 present
df_template['public_setting'] = 1 # 0 private 1 public
df_template['DM_leader'] = 1
df_template['shaking_intensity'] = shake['shaking_intensity']

df_alarm = data[data['Triggers'] == 'Alarm onset']
if len(df_alarm) == 1:
    t_alarm_begin = df_alarm['Begin Time - ss.msec'].values[0]
    df_template.loc[t_alarm_begin, 'alarm_on'] = 1
    df_template['alarm_on'] = df_template['alarm_on'].fillna(method = 'ffill')
else:
    df_template['alarm_on'] = 0
    print('check if there are multiple alarms or no alarm!')


if df_vedio_summary['ObjectsFall'].values[0] != '#NULL!':
    t_obstacle_start = round(float(df_vedio_summary['ObjectsFall'].values[0]))
    df_template.loc[t_obstacle_start, 'obstacle_floor'] = 1
    df_template['obstacle_floor'] = df_template['obstacle_floor'].fillna(method = 'ffill')
else:
    df_template['obstacle_floor'] = 0

df_template['envir_crowded'] = 'crowded'
df_template['start_pos_DM'] = 'standing'

In [80]:
df_template.to_csv('../02/02_Individual_Level_Leader_Annotated_Anchorage_Nestbett_Courthouse_2nd_Floor_Jozie.csv')

# Follower 1

In [85]:
fp_annotation = '../02/02_Individual_Level_Follower_1_Annotated_Anchorage_Nestbett_Courthouse_2nd_Floor_Jozie.txt'
data = read_annotation(fp_annotation)
df_template = read_template(fp_template)

Index(['Begin Time - ss.msec', 'End Time - ss.msec', 'Duration - ss.msec',
       'Transcription', 'Phase', 'Triggers',
       'Responses/Self-protective Actions/Other-protective Actions',
       'Setting: Social and Physical', 'Injuries', 'Visible',
       'Earthquake Attributes', 'Participant 1 (P1)', 'Participant 2 (P2)',
       'Participant 3 (P3)'],
      dtype='object')


In [86]:
df_behavior = data[data['Responses/Self-protective Actions/Other-protective Actions'].isnull() == False]\
                .reset_index()
t_last_behavior_end = df_behavior['End Time - ss.msec'].max()
#df_behavior['End Time - ss.msec'].idxmax(axis = 0)
idx_reserved_max = t_last_behavior_end
df_template = df_template.drop(np.arange(idx_reserved_max, 500))


for i in trange(len(df_behavior)):
    t_curr_state_begin = df_behavior.loc[i, 'Begin Time - ss.msec'] 
    t_curr_state_end = df_behavior.loc[i, 'End Time - ss.msec']
    
    df_template.loc[t_curr_state_begin:t_curr_state_end, 'behavioral_state'] = \
                        df_behavior.loc[i, 'Responses/Self-protective Actions/Other-protective Actions']
    
df_template['behavioral_state'] = df_template['behavioral_state'].fillna(method = 'ffill')


100%|████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<?, ?it/s]


In [87]:
df_template['num_people'] = df_vedio_summary['NumPeople'].values[0]
df_template['cover_availability'] = 1
df_template['child_present'] = 0 # 0 no 1 present
df_template['public_setting'] = 1 # 0 private 1 public
df_template['DM_leader'] = 0 # 0 no 1 leader
df_template['shaking_intensity'] = shake['shaking_intensity']

df_alarm = data[data['Triggers'] == 'Alarm onset']
if len(df_alarm) == 1:
    t_alarm_begin = df_alarm['Begin Time - ss.msec'].values[0]
    df_template.loc[t_alarm_begin, 'alarm_on'] = 1
    df_template['alarm_on'] = df_template['alarm_on'].fillna(method = 'ffill')
else:
    df_template['alarm_on'] = 0
    print('check if there are multiple alarms or no alarm!')


if df_vedio_summary['ObjectsFall'].values[0] != '#NULL!':
    t_obstacle_start = round(float(df_vedio_summary['ObjectsFall'].values[0]))
    df_template.loc[t_obstacle_start, 'obstacle_floor'] = 1
    df_template['obstacle_floor'] = df_template['obstacle_floor'].fillna(method = 'ffill')
else:
    df_template['obstacle_floor'] = 0

df_template['envir_crowded'] = 'crowded'
df_template['start_pos_DM'] = 'standing'

In [88]:
df_template.to_csv('../02/02_Individual_Level_Follower_1_Annotated_Anchorage_Nestbett_Courthouse_2nd_Floor_Jozie.csv')

# Follower 2

In [89]:
fp_annotation = '../02/02_Individual_Level_Follower_2_Annotated_Anchorage_Nestbett_Courthouse_2nd_Floor_Jozie.txt'
data = read_annotation(fp_annotation)
df_template = read_template(fp_template)

Index(['Begin Time - ss.msec', 'End Time - ss.msec', 'Duration - ss.msec',
       'Transcription', 'Phase', 'Triggers',
       'Responses/Self-protective Actions/Other-protective Actions',
       'Setting: Social and Physical', 'Injuries', 'Visible',
       'Earthquake Attributes', 'Participant 1 (P1)', 'Participant 2 (P2)',
       'Participant 3 (P3)'],
      dtype='object')


In [90]:
df_behavior = data[data['Responses/Self-protective Actions/Other-protective Actions'].isnull() == False]\
                .reset_index()
t_last_behavior_end = df_behavior['End Time - ss.msec'].max()
#df_behavior['End Time - ss.msec'].idxmax(axis = 0)
idx_reserved_max = t_last_behavior_end
df_template = df_template.drop(np.arange(idx_reserved_max, 500))


for i in trange(len(df_behavior)):
    t_curr_state_begin = df_behavior.loc[i, 'Begin Time - ss.msec'] 
    t_curr_state_end = df_behavior.loc[i, 'End Time - ss.msec']
    
    df_template.loc[t_curr_state_begin:t_curr_state_end, 'behavioral_state'] = \
                        df_behavior.loc[i, 'Responses/Self-protective Actions/Other-protective Actions']
    
df_template['behavioral_state'] = df_template['behavioral_state'].fillna(method = 'ffill')


100%|██████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 3007.39it/s]


In [91]:
df_template['num_people'] = df_vedio_summary['NumPeople'].values[0]
df_template['cover_availability'] = 1
df_template['child_present'] = 0 # 0 no 1 present
df_template['public_setting'] = 1 # 0 private 1 public
df_template['DM_leader'] = 0 # 0 no 1 leader
df_template['shaking_intensity'] = shake['shaking_intensity']

df_alarm = data[data['Triggers'] == 'Alarm onset']
if len(df_alarm) == 1:
    t_alarm_begin = df_alarm['Begin Time - ss.msec'].values[0]
    df_template.loc[t_alarm_begin, 'alarm_on'] = 1
    df_template['alarm_on'] = df_template['alarm_on'].fillna(method = 'ffill')
else:
    df_template['alarm_on'] = 0
    print('check if there are multiple alarms or no alarm!')


if df_vedio_summary['ObjectsFall'].values[0] != '#NULL!':
    t_obstacle_start = round(float(df_vedio_summary['ObjectsFall'].values[0]))
    df_template.loc[t_obstacle_start, 'obstacle_floor'] = 1
    df_template['obstacle_floor'] = df_template['obstacle_floor'].fillna(method = 'ffill')
else:
    df_template['obstacle_floor'] = 0

df_template['envir_crowded'] = 'crowded'
df_template['start_pos_DM'] = 'standing'

In [92]:
df_template.to_csv('../02/02_Individual_Level_Follower_2_Annotated_Anchorage_Nestbett_Courthouse_2nd_Floor_Jozie.csv')

# Video data integration

In [93]:
df_v02_leader = pd.read_csv('../02/02_Individual_Level_Leader_Annotated_Anchorage_Nestbett_Courthouse_2nd_Floor_Jozie.csv', index_col = 0)
df_v02_follower1 = pd.read_csv('../02/02_Individual_Level_Follower_1_Annotated_Anchorage_Nestbett_Courthouse_2nd_Floor_Jozie.csv',
                             index_col = 0)
df_v02_follower2 = pd.read_csv('../02/02_Individual_Level_Follower_2_Annotated_Anchorage_Nestbett_Courthouse_2nd_Floor_Jozie.csv',
                             index_col = 0)

In [94]:
final_data = df_v02_leader.copy().append(df_v02_follower1).append(df_v02_follower2)

In [98]:
final_data['DM_far_egress'] = 0
final_data['DM_group_size'] = 10 # estimated

In [99]:
final_data = final_data.reset_index().drop('index', axis = 1)
final_data.to_csv('../02/02_Individual_Level_Anchorage_Nestbett_Courthouse_2nd_Floor_Jozie.csv')

# Preprocess descriptive words to numeric values

In [101]:
final_data = pd.read_csv('../02/02_Individual_Level_Anchorage_Nestbett_Courthouse_2nd_Floor_Jozie.csv',
                        index_col = 0)

In [102]:
final_data

,behavioral_state,shaking_intensity,time_elapsed,envir_crowded,cover_availability,obstacle_floor,start_pos_DM,num_people,child_present,public_setting,alarm_on,DM_leader,DM_far_egress,DM_group_size
0,normal,Weak (MMI II),1,crowded,1,0,standing,25,0,1,0.0,1,0,10
1,Other (P1 directs others,Weak (MMI III),2,crowded,1,0,standing,25,0,1,0.0,1,0,10
2,Other (P1 directs others,Weak (MMI III),3,crowded,1,0,standing,25,0,1,1.0,1,0,10
3,Other (P1 directs others),Light (MMI IV),4,crowded,1,0,standing,25,0,1,1.0,1,0,10
4,Other (P1 directs others),Light (MMI IV),5,crowded,1,0,standing,25,0,1,1.0,1,0,10
5,Other (P1 directs others),Light (MMI IV),6,crowded,1,0,standing,25,0,1,1.0,1,0,10
6,Other (P1 directs others),Light (MMI IV),7,crowded,1,0,standing,25,0,1,1.0,1,0,10
7,Other (P1 directs others),Light (MMI IV),8,crowded,1,0,standing,25,0,1,1.0,1,0,10
8,normal,Weak (MMI II),1,crowded,1,0,standing,25,0,1,0.0,0,0,10
9,normal,Weak (MMI III),2,crowded,1,0,standing,25,0,1,0.0,0,0,10


In [18]:
leader = final_data[final_data['DM_leader'] == 1]
nonleader = final_data[final_data['DM_leader'] != 1]

In [10]:
final_data['behavioral_state'].unique()

array(['normal', 'Other (P1 directs others', 'Other (P1 directs others)',
       'P2 drops', 'P2 covers', 'P2 holds', 'P3 other (directs others)',
       'P3 talk on phone onset', 'P3 walk away'], dtype=object)

In [20]:
leader[leader['behavioral_state'].isin(['Other (P1 directs others'])]['time_elapsed'].min()

2

In [21]:
final_data['Being_directed'] = 0

final_data.loc[nonleader[nonleader['time_elapsed']>=3].index, 'Being_directed'] = 1

In [103]:
def find_state(word_string):
    
    splitted_words = word_string.split()
    
    list_cover = ['duck', 'ducks', 'cover', 'covers']
    list_hold_on = ['hold', 'holds', 'tighten', 'tightens']
    list_evacuate = ['evacuate', 'evacuates']

    cover_value = []
    hold_on_value = []
    evacuate_value = []
    
    for word1 in splitted_words:
        if word1 in list_cover:
            cover_value.append(1)
        else:
            cover_value.append(0)
            
        if word1 in list_hold_on:
            hold_on_value.append(1)
        else:
            hold_on_value.append(0)        
            
        if word1 in list_evacuate:
            evacuate_value.append(1)
        else:
            evacuate_value.append(0)            
    
    cover_value = np.array(cover_value)
    hold_on_value = np.array(hold_on_value)    
    evacuate_value = np.array(evacuate_value)    
    
    if sum(cover_value) >= 1:
        return 1
    if sum(hold_on_value) >= 1:
        return 2
    if sum(evacuate_value) >= 1:
        return 3
    if sum(cover_value) == 0 & sum(hold_on_value) == 0 & sum(evacuate_value) == 0:
        return 0

In [104]:
for i in trange(len(final_data)):
    final_data.loc[i, 'behavioral_state'] = find_state(final_data.loc[i, 'behavioral_state'])

100%|████████████████████████████████████████████████████████████████████████████████| 31/31 [00:00<00:00, 4438.27it/s]


In [105]:
def find_crowded(word):
    if word == 'uncrowded':
        return 0
    else:
        return 1

In [106]:
for i in trange(len(final_data)):
    final_data.loc[i, 'envir_crowded'] = find_state(final_data.loc[i, 'envir_crowded'])

100%|████████████████████████████████████████████████████████████████████████████████| 31/31 [00:00<00:00, 5154.34it/s]


In [107]:
print(f"The Other state accounts for {final_data[final_data['behavioral_state'] == 0].shape[0] / len(final_data):.4f}")
print(f"The cover state accounts for {final_data[final_data['behavioral_state'] == 1].shape[0] / len(final_data):.4f}")
print(f"The hold on state accounts for {final_data[final_data['behavioral_state'] == 2].shape[0] / len(final_data):.4f}")
print(f"The evacuate state accounts for {final_data[final_data['behavioral_state'] == 3].shape[0] / len(final_data):.4f}")

The Other state accounts for 0.8387
The cover state accounts for 0.1290
The hold on state accounts for 0.0323
The evacuate state accounts for 0.0000


In [110]:
final_data.to_csv('../Summarized_Data/02.csv')